In [1]:
import sys
sys.path.append('../build/')
%pylab inline
np.set_printoptions(precision=4, suppress=True)
import pandas as pd
import linear_solver
import versor as vsr

Populating the interactive namespace from numpy and matplotlib


In [2]:
# import autograd.numpy as np

from pymanopt.manifolds import manifold
from pymanopt.manifolds import Sphere
from pymanopt import Problem
from pymanopt.solvers import SteepestDescent, TrustRegions, ParticleSwarm, ConjugateGradient, NelderMead
from __future__ import division

import numpy as np
import numpy.linalg as la
import numpy.random as rnd

from pymanopt.manifolds.manifold import Manifold

In [3]:
def create_rotor(th_lims=(0, np.pi/2)):
    rotor = vsr.Rot(vsr.Biv(*np.random.uniform(-1, 1, 3)).unit()
                      * np.random.uniform(*th_lims) * -0.5)
    return rotor

In [4]:
class Motor(Manifold):
    def __init__(self):
        self._shape = 8
        self._name = "Motor manifold"

    def __str__(self):
        return self._name
    
    @property
    def typicaldist(self):
        return np.pi
    
    
    def norm(self, X, U):
        return la.norm(U)
    
    def retr(self, X, U):
        Y = vsr.Mot(*X) + vsr.Mot(*U)
        return np.array(Y.retract())
    
    def CayleySelig(self, X, U):
        M = vsr.Mot(*X)
        B = M.rev() * vsr.Mot(*U)
        
        Rp = vsr.Mot(1.0, B[1], B[2], B[3], 0.0, 0.0, 0.0, 0.0)
        Rn = vsr.Mot(1.0, -B[1], -B[2], -B[3], 0.0, 0.0, 0.0, 0.0)
        Rninv = Rn.inv()
        eps = vsr.Mot(0,0,0,0,0,0,0,-1)
        b = vsr.Mot(0.0, B[6], -B[5], B[4], 0.0, 0.0, 0.0, 0.0)
        return np.array(M * (Rp * Rninv + eps * Rninv * b * Rninv * 2))

    def oexp(self,X,U):

        M = vsr.Mot(*X)
        B = M.rev() * vsr.Mot(*U)
        n = np.sqrt(1 + B[1] * B[1] + B[2] * B[2] + B[3] * B[3])
        s = B[1] * B[6] - B[2] * B[5] + B[3] * B[4]
        m = vsr.Mot(1.0, B[1], B[2], B[3], B[4], B[5], B[6], s) * (1.0 / n)
        return np.array(M * m)

    def CayleyLi(self,X, U):        
        M = vsr.Mot(*X)
        B = M.rev() * vsr.Mot(*U)
        B_ = vsr.Mot(0.0, B[1], B[2], B[3], B[4], B[5], B[6], 0.0)
        BB = B_ * B_
        Rp = vsr.Mot(1.0, B[1], B[2], B[3], B[4], B[5], B[6], 0.0)
        R0 = vsr.Mot(1.0 - BB[0], 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
        R4 = vsr.Mot(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, BB[7])
        Rn = R0 + R4
        Rden = R0 * R0 
        return np.array(M * (Rp * Rp * Rn * Rden.inv()))

    def exp(self, X, U):
        M = vsr.Mot(*X)
        B = M.rev() * vsr.Mot(*U)
        return np.array(M * vsr.Dll(B[1], B[2], B[3], B[4], B[5], B[6]).exp())
    
#     retr = CayleySelig
#     retr = oexp
#     retr = CayleyLi
    retr = exp
    
    def inner(self, X, U, V):
        return float(np.tensordot(U, V, axes=U.ndim))

    def norm(self, X, U):
        return la.norm(U)

    def dist(self, U, V):
        # Make sure inner product is between -1 and 1
        inner = max(min(self.inner(None, U, V), 1), -1)
        return np.arccos(inner)

    def proj(self, X, H):
        return H - self.inner(None, X, H) * X

    egrad2rgrad = proj

    def ehess2rhess(self, X, egrad, ehess, U):
        return self.proj(X, ehess) - self.inner(None, X, egrad) * U


    @property
    def dim(self):
        return 6

In [5]:
class Rotator(Sphere):
    def __init__(self):
        super().__init__(4)
        
    def exp(self, X, U):
        R = vsr.Rot(*X)
        g = R.rev() * vsr.Rot(*U)
        B = vsr.Biv(g[1], g[2], g[3]) * -1.0
        return np.array(R * B.exp())
    
    def cay(self, X, U):
        R = vsr.Rot(*X)
        g = R.rev() * vsr.Rot(*U)
        Y = vsr.Rot(*self._normalize(np.array([1.0, g[1], g[2], g[3]])))
        return np.array(R * Y)
    
    retr = cay
        
        

In [6]:
def create_motor(d_lims=(0, 1), th_lims=(0, np.pi)):
    translator = (vsr.Vec(*np.random.random(3)).unit()
                  * np.random.uniform(*d_lims)).trs()
    rotator = vsr.Rot(vsr.Biv(*np.random.uniform(-1, 1, 3)).unit()
                      * np.random.uniform(*th_lims) * -0.5)
    motor = translator * rotator
    return motor

In [7]:
n_points=10

m0 = create_motor(th_lims=(0,np.pi/3))
points_a = [vsr.Vec(*np.random.normal(0.0, 0.8, 3)).null() for i in range(n_points)]
points_b = [point.spin(m0) for point in points_a]

In [8]:
def grade2(Y):
    Y[0] = 0.0
    Y[26] = 0.0
    Y[27] = 0.0
    Y[28] = 0.0
    Y[29] = 0.0
    Y[30] = 0.0
    return Y

In [9]:
def proj(X):
    M = vsr.Mot(0,0,0,0,0,0,0,0)
    sgns = [1,2,3,7]
    for i in range(8):
        Ei = vsr.MotRec(0,0,0,0,0,0,0,0)
        if i in sgns:
            Ei[i] = -1.0
        else:
            Ei[i] = 1.0
            
        Ei[i] = 1.0
        M[i] = (X * vsr.CGA(Ei))[0]
    return vsr.CGA(M)

In [10]:
def adproj(X):
    M = vsr.MotRec(0,0,0,0,0,0,0,0)
    sgns = [1,2,3,7]
    for i in range(8):
        Ei = vsr.Mot(0,0,0,0,0,0,0,0)
        if i in sgns:
            Ei[i] = -1.0
        else:
            Ei[i] = 1.0
            
        M[i] = (vsr.CGA(Ei) * X)[0]
    return vsr.CGA(M)

In [11]:
A = vsr.CGA(points_a[0])
B = vsr.CGA(points_b[0])
Y = adproj(grade2((A * M.rev() * B + A.rev() * M.rev() * B.rev()) * M))
G = np.array([(Y * vsr.CGA(vsr.Mot(0,1,0,0,0,0,0,0)))[0],
(Y * vsr.CGA(vsr.Mot(0,0,1,0,0,0,0,0)))[0],
(Y * vsr.CGA(vsr.Mot(0,0,0,1,0,0,0,0)))[0],
(Y * vsr.CGA(vsr.Mot(0,0,0,0,1,0,0,0)))[0],
(Y * vsr.CGA(vsr.Mot(0,0,0,0,0,1,0,0)))[0],
(Y * vsr.CGA(vsr.Mot(0,0,0,0,0,0,1,0)))[0]]).reshape(6)
G

NameError: name 'M' is not defined

In [12]:
man = Sphere(4)

In [13]:
R = create_rotor()

In [14]:
G = vsr.Rot(1,2,3,4)

In [15]:
man.proj(np.array(R), np.array(G))

array([ 0.1317,  2.3905,  3.3176,  3.4585])

In [16]:
Y = (G * R).biv() * R.rev()
Y

Rot: [ 0.71 1.9 2.9 4.2 ]

In [17]:
Y = R * (R.rev() * G).biv()
np.array(Y)

array([ 0.1317,  2.3905,  3.3176,  3.4585])

In [18]:
Y = adproj(M *  grade2(M.rev() * (A * M.rev() * B + A.rev() * M.rev() * B.rev())))
Y

NameError: name 'M' is not defined

In [19]:
proj(Y)

TypeError: __mul__(): incompatible function arguments. The following argument types are supported:
    1. (self: versor_pybind11.Rot, arg0: float) -> versor_pybind11.Rot
    2. (self: versor_pybind11.Rot, arg0: versor_pybind11.Biv) -> versor_pybind11.Rot
    3. (self: versor_pybind11.Rot, arg0: vsr::Multivector<vsr::algebra<vsr::metric<4, 1, true>, double>, vsr::Basis<(short)0, (short)17, (short)18, (short)20> >) -> vsr::Multivector<vsr::algebra<vsr::metric<4, 1, true>, double>, vsr::Basis<(short)0, (short)3, (short)5, (short)6, (short)17, (short)18, (short)20, (short)23> >
    4. (self: versor_pybind11.Rot, arg0: versor_pybind11.Rot) -> versor_pybind11.Rot

Invoked with: Rot: [ 0.13 2.4 3.3 3.5 ], CGA: [ 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ]

In [20]:
adproj(Y)

TypeError: __mul__(): incompatible function arguments. The following argument types are supported:
    1. (self: versor_pybind11.CGA, arg0: float) -> versor_pybind11.CGA
    2. (self: versor_pybind11.CGA, arg0: versor_pybind11.CGA) -> versor_pybind11.CGA

Invoked with: CGA: [ 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ], Rot: [ 0.13 2.4 3.3 3.5 ]

In [21]:
np.array(vsr.Mot(proj((M.rev() * vsr.CGA(vsr.Mot(*gradk3(points_a[0], points_b[0], np.array(vsr.Mot(M)))))))).dll())

NameError: name 'M' is not defined

In [22]:
def rotor_rotate_vector(a, X):
    a1, a2, a3 = np.array(a)
    r0, r1, r2, r3 = np.array(X)
    return (a1 * r3 * r3 + (2.0 * a3 * r1 - 2.0 * a2 * r2) * r3 - a1 * r2 * r2 + 2.0 * a3 * r0 * r2 - a1 * r1 * r1 + 2.0 * a2 * r0 * r1 + a1 * r0 * r0,
            (-a2) * r3 * r3 + (2.0 * a3 * r0 - 2.0 * a1 * r2) * r3 + a2 * r2 * r2 - 2.0 * a3 * r1 * r2 - a2 * r1 * r1 - 2.0 * a1 * r0 * r1 + a2 * r0 * r0,
            (-a3) * r3 * r3 + (2.0 * a1 * r1 - 2.0 * a2 * r0) * r3 - a3 * r2 * r2 + (-2.0 * a2 * r1 - 2.0 * a1 * r0) * r2 + a3 * r1 * r1 + a3 * r0 * r0)

In [23]:
def gradk(a, b, X):
    a1, a2, a3 = np.array(a)
    b1, b2, b3 = np.array(b)
    r0, r1, r2, r3 = np.array(X)
    return np.array(vsr.Rot(r0, r1, r2, r3) * vsr.Biv(
        (-2.0 * a1 * b2 - 2.0 * a2 * b1) * r3 * r3 + ((4.0 * a2 * b2 - 4.0 * a1 * b1) * r2 - 4.0 * a2 * b3 * r1 - 4.0 * a1 * b3 * r0) * r3 + (2.0 * a1 * b2 + 2.0 * a2 * b1) * r2 * r2 + (4.0 * a2 * b3 * r0 - 4.0 * a1 * b3 * r1) * r2 + (2.0 * a2 * b1 - 2.0 * a1 * b2) * r1 * r1 + (4.0 * a2 * b2 + 4.0 * a1 * b1) * r0 * r1 + (2.0 * a1 * b2 - 2.0 * a2 * b1) * r0 * r0,
        (-2.0 * a1 * b3 - 2.0 * a3 * b1) * r3 * r3 + (4.0 * a3 * b2 * r2 + (4.0 * a1 * b1 - 4.0 * a3 * b3) * r1 + 4.0 * a1 * b2 * r0) * r3 + (2.0 * a3 * b1 - 2.0 * a1 * b3) * r2 * r2 + ((4.0 * a3 * b3 + 4.0 * a1 * b1) * r0 - 4.0 * a1 * b2 * r1) * r2 + (2.0 * a1 * b3 + 2.0 * a3 * b1) * r1 * r1 + 4.0 * a3 * b2 * r0 * r1 + (2.0 * a1 * b3 - 2.0 * a3 * b1) * r0 * r0,
        (2.0 * a3 * b2 - 2.0 * a2 * b3) * r3 * r3 + (4.0 * a3 * b1 * r2 + 4.0 * a2 * b1 * r1 + (4.0 * a3 * b3 + 4.0 * a2 * b2) * r0) * r3 + (-2.0 * a2 * b3 - 2.0 * a3 * b2) * r2 * r2 + ((4.0 * a3 * b3 - 4.0 * a2 * b2) * r1 + 4.0 * a2 * b1 * r0) * r2 + (2.0 * a2 * b3 + 2.0 * a3 * b2) * r1 * r1 - 4.0 * a3 * b1 * r0 * r1 + (2.0 * a2 * b3 - 2.0 * a3 * b2) * r0 * r0))

In [24]:
def grad_explicit(X): 
    g = np.sum([gradk(a, b, X) for a, b in zip(points_a, points_b)], axis=0) * 2.0
    return g

In [25]:
def costk3(a, b, X):
    a1, a2, a3, a4, a5 = np.array(a)
    b1, b2, b3, b4, b5 = np.array(b)
    m1, m2, m3, m4, m5, m6, m7, m8 = X
    return 4.0 * a4 * b4 * m8 * m8 + ((4.0 * a4 * b1 - 4.0 * a1 * b4) * m4 + (4.0 * a2 * b4 - 4.0 * a4 * b2) * m3 + (4.0 * a4 * b3 - 4.0 * a3 * b4) * m2) * m8 + 4.0 * a4 * b4 * m7 * m7 + ((4.0 * a2 * b4 + 4.0 * a4 * b2) * m4 + (4.0 * a1 * b4 + 4.0 * a4 * b1) * m3 + (4.0 * a4 * b3 - 4.0 * a3 * b4) * m1) * m7 + 4.0 * a4 * b4 * m6 * m6 + ((-4.0 * a3 * b4 - 4.0 * a4 * b3) * m4 + (4.0 * a1 * b4 + 4.0 * a4 * b1) * m2 + (4.0 * a4 * b2 - 4.0 * a2 * b4) * m1) * m6 + 4.0 * a4 * b4 * m5 * m5 + ((-4.0 * a3 * b4 - 4.0 * a4 * b3) * m3 + (-4.0 * a2 * b4 - 4.0 * a4 * b2) * m2 + (4.0 * a4 * b1 - 4.0 * a1 * b4) * m1) * m5 + (2.0 * a4 * b5 + 2.0 * a5 * b4 + 2.0 * a3 * b3 + 2.0 * a2 * b2 - 2.0 * a1 * b1) * m4 * m4 + ((4.0 * a1 * b2 + 4.0 * a2 * b1) * m3 + (-4.0 * a1 * b3 - 4.0 * a3 * b1) * m2 + (4.0 * a2 * b3 - 4.0 * a3 * b2) * m1) * m4 + (2.0 * a4 * b5 + 2.0 * a5 * b4 + 2.0 * a3 * b3 - 2.0 * a2 * b2 + 2.0 * a1 * b1) * m3 * m3 + ((4.0 * a2 * b3 + 4.0 * a3 * b2) * m2 + (4.0 * a1 * b3 - 4.0 * a3 * b1) * m1) * m3 + (2.0 * a4 * b5 + 2.0 * a5 * b4 - 2.0 * a3 * b3 + 2.0 * a2 * b2 + 2.0 * a1 * b1) * m2 * m2 + (4.0 * a1 * b2 - 4.0 * a2 * b1) * m1 * m2 + (2.0 * a4 * b5 + 2.0 * a5 * b4 - 2.0 * a3 * b3 - 2.0 * a2 * b2 - 2.0 * a1 * b1) * m1 * m1

In [26]:
def cost3(X): 
    return np.sum([costk3(a, b, X) for a, b in zip(points_a, points_b)])

In [27]:
def gradk3(a,b,X):
    a1, a2, a3, a4, a5 = np.array(a)
    b1, b2, b3, b4, b5 = np.array(b)
    m1, m2, m3, m4, m5, m6, m7, m8 = X
    B0 = (-4.0 * a2 * b4 * m4 - 4.0 * a1 * b4 * m3) * m8 + (4.0 * a2 * b4 * m3 - 4.0 * a1 * b4 * m4) * m7 + (4.0 * a2 * b4 * m2 + 4.0 * a1 * b4 * m1) * m6 + (4.0 * a1 * b4 * m2 - 4.0 * a2 * b4 * m1) * m5 + (-2.0 * a1 * b2 - 2.0 * a2 * b1) * m4 * m4 + ((4.0 * a2 * b2 - 4.0 * a1 * b1) * m3 - 4.0 * a2 * b3 * m2 - 4.0 * a1 * b3 * m1) * m4 + (2.0 * a1 * b2 + 2.0 * a2 * b1) * m3 * m3 + (4.0 * a2 * b3 * m1 - 4.0 * a1 * b3 * m2) * m3 + (2.0 * a2 * b1 - 2.0 * a1 * b2) * m2 * m2 + (4.0 * a2 * b2 + 4.0 * a1 * b1) * m1 * m2 + (2.0 * a1 * b2 - 2.0 * a2 * b1) * m1 * m1
    B1 = (4.0 * a1 * b4 * m2 - 4.0 * a3 * b4 * m4) * m8 + (4.0 * a3 * b4 * m3 + 4.0 * a1 * b4 * m1) * m7 + (4.0 * a1 * b4 * m4 + 4.0 * a3 * b4 * m2) * m6 + (4.0 * a1 * b4 * m3 - 4.0 * a3 * b4 * m1) * m5 + (-2.0 * a1 * b3 - 2.0 * a3 * b1) * m4 * m4 + (4.0 * a3 * b2 * m3 + (4.0 * a1 * b1 - 4.0 * a3 * b3) * m2 + 4.0 * a1 * b2 * m1) * m4 + (2.0 * a3 * b1 - 2.0 * a1 * b3) * m3 * m3 + ((4.0 * a3 * b3 + 4.0 * a1 * b1) * m1 - 4.0 * a1 * b2 * m2) * m3 + (2.0 * a1 * b3 + 2.0 * a3 * b1) * m2 * m2 + 4.0 * a3 * b2 * m1 * m2 + (2.0 * a1 * b3 - 2.0 * a3 * b1) * m1 * m1
    B2 = (4.0 * a3 * b4 * m3 + 4.0 * a2 * b4 * m2) * m8 + (4.0 * a3 * b4 * m4 + 4.0 * a2 * b4 * m1) * m7 + (4.0 * a2 * b4 * m4 - 4.0 * a3 * b4 * m1) * m6 + (4.0 * a2 * b4 * m3 - 4.0 * a3 * b4 * m2) * m5 + (2.0 * a3 * b2 - 2.0 * a2 * b3) * m4 * m4 + (4.0 * a3 * b1 * m3 + 4.0 * a2 * b1 * m2 + (4.0 * a3 * b3 + 4.0 * a2 * b2) * m1) * m4 + (-2.0 * a2 * b3 - 2.0 * a3 * b2) * m3 * m3 + ((4.0 * a3 * b3 - 4.0 * a2 * b2) * m2 + 4.0 * a2 * b1 * m1) * m3 + (2.0 * a2 * b3 + 2.0 * a3 * b2) * m2 * m2 - 4.0 * a3 * b1 * m1 * m2 + (2.0 * a2 * b3 - 2.0 * a3 * b2) * m1 * m1
    B3 = 4.0 * a4 * b4 * m4 * m8 - 4.0 * a4 * b4 * m3 * m7 - 4.0 * a4 * b4 * m2 * m6 + 4.0 * a4 * b4 * m1 * m5 + (2.0 * a4 * b1 - 2.0 * a1 * b4) * m4 * m4 + (4.0 * a4 * b3 * m2 - 4.0 * a4 * b2 * m3) * m4 + (-2.0 * a1 * b4 - 2.0 * a4 * b1) * m3 * m3 - 4.0 * a4 * b3 * m1 * m3 + (-2.0 * a1 * b4 - 2.0 * a4 * b1) * m2 * m2 - 4.0 * a4 * b2 * m1 * m2 + (2.0 * a4 * b1 - 2.0 * a1 * b4) * m1 * m1
    B4 = -4.0 * a4 * b4 * m3 * m8 - 4.0 * a4 * b4 * m4 * m7 + 4.0 * a4 * b4 * m1 * m6 + 4.0 * a4 * b4 * m2 * m5 + (-2.0 * a2 * b4 - 2.0 * a4 * b2) * m4 * m4 + (-4.0 * a4 * b1 * m3 - 4.0 * a4 * b3 * m1) * m4 + (2.0 * a4 * b2 - 2.0 * a2 * b4) * m3 * m3 - 4.0 * a4 * b3 * m2 * m3 + (-2.0 * a2 * b4 - 2.0 * a4 * b2) * m2 * m2 + 4.0 * a4 * b1 * m1 * m2 + (2.0 * a4 * b2 - 2.0 * a2 * b4) * m1 * m1
    B5 = 4.0 * a4 * b4 * m2 * m8 + 4.0 * a4 * b4 * m1 * m7 + 4.0 * a4 * b4 * m4 * m6 + 4.0 * a4 * b4 * m3 * m5 + (-2.0 * a3 * b4 - 2.0 * a4 * b3) * m4 * m4 + (4.0 * a4 * b1 * m2 + 4.0 * a4 * b2 * m1) * m4 + (-2.0 * a3 * b4 - 2.0 * a4 * b3) * m3 * m3 + (4.0 * a4 * b1 * m1 - 4.0 * a4 * b2 * m2) * m3 + (2.0 * a4 * b3 - 2.0 * a3 * b4) * m2 * m2 + (2.0 * a4 * b3 - 2.0 * a3 * b4) * m1 * m1
    return np.array(vsr.Mot(m1, m2, m3, m4, m5, m6, m7, m8) * vsr.Dll(B0, B1, B2, B3, B4, B5))

In [28]:
def grad_explicit3(X): 
    g = np.sum([gradk3(a, b, X) for a, b in zip(points_a, points_b)], axis=0) * 2.0
    return g

In [29]:
def costk(a, b, X): 
    a1, a2, a3 = np.array(a)
    b1, b2, b3 = np.array(b)
    r0, r1, r2, r3 = X
    o0 = a1 * r3 * r3 + (2.0 * a3 * r1 - 2.0 * a2 * r2) * r3 - a1 * r2 * r2 + 2.0 * a3 * r0 * r2 - a1 * r1 * r1 + 2.0 * a2 * r0 * r1 + a1 * r0 * r0 - b1
    o1 = (-a2) * r3 * r3 + (2.0 * a3 * r0 - 2.0 * a1 * r2) * r3 + a2 * r2 * r2 - 2.0 * a3 * r1 * r2 - a2 * r1 * r1 - 2.0 * a1 * r0 * r1 + a2 * r0 * r0 - b2
    o2 = (-a3) * r3 * r3 + (2.0 * a1 * r1 - 2.0 * a2 * r0) * r3 - a3 * r2 * r2 + (-2.0 * a2 * r1 - 2.0 * a1 * r0) * r2 + a3 * r1 * r1 + a3 * r0 * r0 - b3
    return o0 * o0 + o1 * o1 + o2 * o2

def costk2(a, b, X):
    a1, a2, a3 = np.array(a)
    b1, b2, b3 = np.array(b)
    r0, r1, r2, r3 = X
    return ((a3 * b3 + a2 * b2 - a1 * b1) * r3 * r3 +
            ((2.0 * a1 * b2 + 2.0 * a2 * b1) * r2 + 
             (-2.0 * a1 * b3 - 2.0 * a3 * b1) * r1 + 
             (2.0 * a2 * b3 - 2.0 * a3 * b2) * r0) * r3 + 
            (a3 * b3 - a2 * b2 + a1 * b1) * r2 * r2 + 
            ((2.0 * a2 * b3 + 2.0 * a3 * b2) * r1 + 
             (2.0 * a1 * b3 - 2.0 * a3 * b1) * r0) * r2 + 
            ((-a3) * b3 + a2 * b2 + a1 * b1) * r1 * r1 + 
            (2.0 * a1 * b2 - 2.0 * a2 * b1) * r0 * r1 + 
            ((-a3) * b3 - a2 * b2 - a1 * b1) * r0 * r0)

# (2) Define the cost function (here using autograd.numpy)
def cost(X): 
    return np.sum([costk(a, b, X) for a, b in zip(points_a, points_b)])

def cost2(X): 
    return np.sum([costk2(a, b, X) for a, b in zip(points_a, points_b)])

In [31]:
n_points=3

m0 = create_motor()
points_a = [vsr.Vec(*np.random.normal(0.0, 0.8, 3)).null() for i in range(n_points)]
points_b = [point.spin(m0) for point in points_a]
manifold = Motor()
print(manifold.dim)
problem = Problem(manifold=manifold, cost=cost3, grad=grad_explicit3, verbosity=3)


# (3) Instantiate a Pymanopt solver
solver = SteepestDescent()
# solver = TrustRegions()


# let Pymanopt do the rest
Xopt = solver.solve(problem, x=np.array([1.,0.,0.,0.,0.,0.,0.,0.]))
print(vsr.Mot(*Xopt))
print(la.norm(vsr.Mot(*Xopt).rev() * m0))

6
Compiling cost function...
 iter		   cost val	    grad. norm
    1	+4.8836278962018032e-01	5.32552964e+00
    2	+2.0732824363512045e-01	3.33158094e+00
    3	+1.2028010993209523e-01	2.58408853e+00
    4	+7.6998686717022868e-02	2.08019689e+00
    5	+1.8662359429796160e-02	8.66444439e-01
    6	+7.2113992563322682e-03	3.67252423e-01
    7	+7.2043334816684346e-03	5.72656285e-01
    8	+7.1809206418221583e-03	5.71445034e-01
    9	+7.1035205905779497e-03	5.67423250e-01
   10	+6.7881963593107442e-03	5.49443350e-01
   11	+5.5729898469919770e-03	4.74272540e-01
   12	+2.3109303561692218e-03	1.66555694e-01
   13	+1.5895376953105611e-03	1.50977198e-01
   14	+1.2177931319753421e-03	1.58729843e-01
   15	+9.8636666358856151e-04	1.61150675e-01
   16	+5.6385570450614986e-04	6.17597174e-02
   17	+4.5642581712684871e-04	1.08985575e-01
   18	+2.6765586650676076e-04	4.14215447e-02
   19	+2.3481692546897803e-04	7.89167637e-02
   20	+1.4595185859900273e-04	3.08002178e-02
   21	+1.2173996606354431e-04	4.84866

In [33]:
n_points=3

m0 = create_rotor()
points_a = [vsr.Vec(*np.random.normal(0.0, 0.8, 3)) for i in range(n_points)]
points_b = [point.spin(m0) for point in points_a]

manifold = Sphere(4)
problem = Problem(manifold=manifold, cost=cost, grad=grad_explicit, verbosity=1)

# (3) Instantiate a Pymanopt solver
# solver = SteepestDescent()
solver = TrustRegions()
# solver = ParticleSwarm()
# solver = ConjugateGradient()
# solver = NelderMead()


# let Pymanopt do the rest
Xopt = solver.solve(problem, x=np.array([1.,0.,0.,0.]))
print(vsr.Rot(*Xopt))
print(vsr.Rot(*Xopt).rev() * m0)

Compiling cost function...
Computing Hessian of cost function...
Optimizing...
Computing gradient of cost function...
[ -8.9904 -26.8566 -83.9793  51.1651]
[-0.0954  8.6664 -2.248  -0.734 ]
[ 0.0735 -3.1659  3.1651  2.3594]
[ 0.0004  1.4385  0.2334  0.3157]
[-0.0573 -0.175  -0.4718  0.3998]
[-0.      0.0001 -0.0001 -0.0001]
[ 0.     -0.0002  0.0001 -0.    ]
[ 0.  0.  0. -0.]
Terminated - min grad norm reached after 3 iterations, 0.25 seconds.

Rot: [ 1 -0.0021 -0.07 0.06 ]
Rot: [ 1 -1.5e-17 4e-18 -1.4e-17 ]


In [52]:
manifold = Sphere(4)
problem = Problem(manifold=manifold, cost=cost, verbosity=1)

# (3) Instantiate a Pymanopt solver
# solver = SteepestDescent()
# solver = TrustRegions()
# solver = ParticleSwarm()
solver = ConjugateGradient()
# solver = NelderMead()


# let Pymanopt do the rest
Xopt = solver.solve(problem, x=np.array([1.,0.,0.,0.]))
print(vsr.Rot(*Xopt))
print(vsr.Rot(*Xopt).rev() * m0)

Compiling cost function...
Computing gradient of cost function...
Optimizing...
Terminated - min grad norm reached after 27 iterations, 0.18 seconds.

Rot: [ 0.92 0.35 0.18 -0.08 ]
Rot: [ 1 2.7e-08 3.4e-08 -2.5e-08 ]


In [33]:
Rtest = create_rotor()


In [60]:
grad_explicit(np.array(Rtest))

array([-1.6031, -4.8323,  2.0965, -0.8448])

In [61]:
np.dot(np.array(R).T, G)

ValueError: shapes (4,) and (8,) not aligned: 4 (dim 0) != 8 (dim 0)

In [62]:
Y = vsr.EGA(*np.sum([np.array(vsr.EGA(a) * vsr.EGA(Rtest).rev() * vsr.EGA(b) +
 vsr.EGA(a).rev() * vsr.EGA(Rtest.rev()) * vsr.EGA(b).rev()) for a, b in zip(points_a, points_b)], axis=0))

In [63]:
Y_ = Y * vsr.EGA(Rtest)
Y_[0] = 0
G = Y_ * vsr.EGA(Rtest.rev()) * -2
# G[1] * -1
# G[2] * -1
# G[3] * -1

In [64]:
np.array(G.rev())

array([-1.6031, -0.    , -0.    , -0.    , -4.8323,  2.0965, -0.8448,  0.    ])

In [66]:
np.dot(np.array(G.rev()), np.array(vsr.EGA(Rtest)))

-2.7755575615628914e-17